In [1]:
import pandas as pd
import numpy as np
import datatable as dt

### Import Data

In [2]:
Accident = dt.fread("/Users/user/Desktop/自動化/dft-accident-data/Accidents0515.csv").to_pandas()
Casualty = dt.fread("/Users/user/Desktop/自動化/dft-accident-data/Clean Data/Casualties.csv").to_pandas()
Vehicle = dt.fread("/Users/user/Desktop/自動化/dft-accident-data/Clean Data/Vehicles.csv").to_pandas()

### Merge the Three Raw Datas

In [3]:
Total = Accident.merge(Casualty, how = "outer").merge(Vehicle, how = "left")

In [4]:
# Total.shape

In [5]:
# Check Missing
# Total.isnull().sum()[Total.isnull().sum() != 0]

### Replace Values with Relational Datasets

In [6]:
def label_map(column_name,file) :
    df = pd.read_csv("/Users/user/Desktop/自動化/dft-accident-data/contextCSVs/" + file)
    DICT = dict(zip(df[df.columns[0]], df[df.columns[1]]))
    global Total
    Total[column_name] = Total[column_name].map(DICT)

In [7]:
label_map("Age_Band_of_Driver","Age_Band.csv")   #Age_Band
label_map("Age_Band_of_Casualty","Age_Band.csv") #Age_Band
label_map("Casualty_Class","Casualty_Class.csv")
label_map("Casualty_Type","Casualty_Type.csv")
label_map("Day_of_Week", "Day_of_Week.csv")
label_map("Journey_Purpose_of_Driver", "Journey_Purpose.csv")
label_map("Junction_Control", "Junction_Control.csv")
label_map("Junction_Detail", "Junction_Detail.csv")
label_map("Junction_Location", "Junction_Location.csv")
label_map("Light_Conditions", "Light_Conditions.csv")
label_map("Local_Authority_(District)", "Local_Authority_District.csv")
label_map("Local_Authority_(Highway)", "Local_Authority_Highway.csv")
label_map("Pedestrian_Crossing-Human_Control", "Ped_Cross_Human.csv")
label_map("Pedestrian_Crossing-Physical_Facilities", "Ped_Cross_Physical.csv")
label_map("Pedestrian_Location", "Ped_Location.csv")
label_map("Pedestrian_Movement", "Ped_Movement.csv")
label_map("1st_Point_of_Impact", "Point_of_Impact.csv")
label_map("Police_Force","Police_Force.csv")
label_map("Did_Police_Officer_Attend_Scene_of_Accident", "Police_Officer_Attend.csv")
label_map("1st_Road_Class", "Road_Class.csv") #Road_Class
label_map("2nd_Road_Class", "Road_Class.csv") #Road_Class
label_map("Road_Type", "Road_Type.csv")
label_map("Sex_of_Driver", "Sex_of_Driver.csv")
label_map("Sex_of_Casualty", "Sex_of_Driver.csv")
label_map("Urban_or_Rural_Area", "Urban_Rural.csv")
label_map("Vehicle_Location-Restricted_Lane", "Vehicle_Location.csv")
label_map("Vehicle_Manoeuvre", "Vehicle_Manoeuvre.csv")
label_map("Vehicle_Type", "Vehicle_Type.csv")

### Create Response Term

In [8]:
# code,label
# 1,Fatal
# 2,Serious
# 3,Slight

Total["Diff_ASCS"] = Total.Accident_Severity - Total.Casualty_Severity
Total["LuckyOrNot"] = Total.Diff_ASCS.map({0:"UnLucky", -1: "Lucky", -2 : "Lucky"})
Total.drop(["Diff_ASCS", "Accident_Severity", "Casualty_Severity"], axis = 1, inplace = True)
# Total.head()

### Drop No Use Columns

In [9]:
Total.drop(["Accident_Index", "Time", "Date", 
            "Location_Easting_OSGR",
            "Location_Northing_OSGR", "Longitude", "Latitude"], axis = 1, inplace = True) 

### Confirm Column Types

In [10]:
# 需要用到的變數並且分類成類別或是數值
DiscreteVar = ["Police_Force", "Day_of_Week", "1st_Road_Class", "Road_Type", "Vehicle_Location-Restricted_Lane",
               "Junction_Detail", "Junction_Control", "2nd_Road_Class", "Was_Vehicle_Left_Hand_Drive?",
               "Pedestrian_Crossing-Physical_Facilities", "Light_Conditions", "Weather_Conditions",
               "Road_Surface_Conditions", "Special_Conditions_at_Site", "Carriageway_Hazards", "Journey_Purpose_of_Driver",
               "Urban_or_Rural_Area", "Did_Police_Officer_Attend_Scene_of_Accident", "Casualty_Class", 
               "Sex_of_Casualty", "Age_Band_of_Casualty", "Casualty_Type", "Pedestrian_Crossing-Human_Control", 
               "Pedestrian_Location", "Pedestrian_Movement", "Casualty_Home_Area_Type", "Vehicle_Type", 
               "Towing_and_Articulation", "Vehicle_Manoeuvre" ,"Junction_Location", "Skidding_and_Overturning",
               "Hit_Object_in_Carriageway", "1st_Point_of_Impact", "Sex_of_Driver", "Age_Band_of_Driver", 
               "Propulsion_Code", "Driver_IMD_Decile", "Driver_Home_Area_Type", "LuckyOrNot"]

NumericVar = ["Number_of_Vehicles", "Number_of_Casualties", "Speed_limit", "Age_of_Casualty", 
              "Car_Passenger", "Bus_or_Coach_Passenger", "Pedestrian_Road_Maintenance_Worker",
              "Age_of_Driver", "Engine_Capacity_(CC)", "Age_of_Vehicle"]

In [11]:
DF = Total[DiscreteVar + NumericVar]

In [12]:
pd.options.mode.chained_assignment = None

In [13]:
def COL_DROP_RET():
    ser = DF[DiscreteVar].apply(lambda x : len(x.value_counts()))
    ColumnsToDrop = list(ser.index[ser >= 25])
    return ColumnsToDrop

drop_list = COL_DROP_RET()
DF.drop(labels = drop_list, axis = 1, inplace = True)

### Replace Missing Values to -999

In [14]:
NA_Labels = [-1, "Unknown", "", "nan", "Total missing or out of range" ,
             "Other/Not known (2005-10)", "Data missing or out of range",
             "data missing", np.nan, "Not known"]
DF.replace(to_replace = NA_Labels, value = -999, inplace = True)

### Output Data

In [16]:
DF.shape

(2402909, 48)

In [17]:
DF.head()

,Day_of_Week,1st_Road_Class,Road_Type,Vehicle_Location-Restricted_Lane,Junction_Detail,Junction_Control,2nd_Road_Class,Was_Vehicle_Left_Hand_Drive?,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,...,Number_of_Vehicles,Number_of_Casualties,Speed_limit,Age_of_Casualty,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle
0,Tuesday,A,Single carriageway,On main c'way - not in restricted lane,Not at junction or within 20 metres,-999,-999,1,Zebra,Daylight,...,1,1,30,37,0,0,-999,74,-999,-999
1,Wednesday,B,Dual carriageway,On main c'way - not in restricted lane,Crossroads,Auto traffic signal,C,1,Pedestrian phase at traffic signal junction,Darkness - lights lit,...,1,1,30,37,0,4,-999,42,8268,3
2,Thursday,C,Single carriageway,On main c'way - not in restricted lane,Not at junction or within 20 metres,-999,-999,1,No physical crossing facilities within 50 metres,Darkness - lights lit,...,2,1,30,62,0,0,-999,62,1762,6
3,Friday,A,Single carriageway,On main c'way - not in restricted lane,Not at junction or within 20 metres,-999,-999,1,No physical crossing facilities within 50 metres,Daylight,...,1,1,30,30,0,0,-999,49,1769,4
4,Monday,Unclassified,Single carriageway,On main c'way - not in restricted lane,Not at junction or within 20 metres,-999,-999,1,No physical crossing facilities within 50 metres,Darkness - lighting unknown,...,1,1,30,49,0,0,-999,49,85,10


In [16]:
DF.to_csv("/Users/user/Desktop/自動化/Response Data.csv", index = False)